# Kaggle - Titanic

In [28]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [29]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
data = pd.read_csv('./data/gender_submission.csv')

train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [37]:
mean_age = np.nanmean(list(train["Age"]))
max_age = np.amax(list(train["Age"]))
train = train.fillna(mean_age)


mean_n_of_family = np.mean([row['SibSp'] + row['Parch'] for index, row in train.iterrows()])

In [38]:
def gender_to_int(gender):
    return 1 if gender == 'male' else 0

def embarked_to_int(embarked):
    if embarked == 'S':
        return 0
    elif embarked == 'C': 
        return 1
    else:
        return 2
    
def get_x_data(data):
    return [ [gender_to_int(row['Sex']), row['Age']/max_age, row['Pclass'], (row['SibSp'] + row['Parch'])/mean_n_of_family] for index, row in data.iterrows() ]


X = get_x_data(train)
Y = list(train["Survived"])

In [39]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

## Using a RandomForestClassifier

In [40]:
rd = RandomForestClassifier()

rd.fit(x_train, y_train)

//anaconda3/envs/kaggle-titanic/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [41]:
rd.score(x_test, y_test)

0.8305084745762712

In [42]:
rd.feature_importances_

array([0.30862016, 0.44007973, 0.14430759, 0.10699252])

In [71]:
test = test.fillna(mean_age)
x_test_data = get_x_data(test)

y_pred = rd.predict(x_test_data)

final = pd.DataFrame(zip(test['PassengerId'], y_pred), columns=['PassengerId', 'Survived'])

In [44]:
final.to_csv('result_rd.csv', index=False)

## Using a KNeighborsClassifier

In [45]:
from sklearn.neighbors import KNeighborsClassifier

In [47]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [48]:
knn.score(x_test, y_test)

0.8067796610169492

## Using a basic neural network

In [49]:
from keras import Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.losses import binary_crossentropy

In [63]:
lr = 0.001

model = Sequential()
model.add(Dense(8, input_shape=(4,), activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=binary_crossentropy,
              optimizer=Adam(lr),
             metrics=['acc'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 8)                 40        
_________________________________________________________________
dense_9 (Dense)              (None, 16)                144       
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 17        
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


In [64]:
model.fit(np.array(x_train), y_train,
                  epochs=100,
                  batch_size=5,
                  validation_data=(np.array(x_test), y_test))

Train on 596 samples, validate on 295 samples
Epoch 1/100
596/596 [==============================] - 1s 2ms/step - loss: 0.7100 - acc: 0.4698 - val_loss: 0.6603 - val_acc: 0.5932
Epoch 2/100
596/596 [==============================] - 0s 389us/step - loss: 0.6274 - acc: 0.6275 - val_loss: 0.6202 - val_acc: 0.5932
Epoch 3/100
596/596 [==============================] - 0s 433us/step - loss: 0.5982 - acc: 0.6292 - val_loss: 0.5983 - val_acc: 0.6068
Epoch 4/100
596/596 [==============================] - 0s 363us/step - loss: 0.5793 - acc: 0.6829 - val_loss: 0.5766 - val_acc: 0.6983
Epoch 5/100
596/596 [==============================] - 0s 377us/step - loss: 0.5573 - acc: 0.7114 - val_loss: 0.5482 - val_acc: 0.7322
Epoch 6/100
596/596 [==============================] - 0s 662us/step - loss: 0.5370 - acc: 0.7550 - val_loss: 0.5275 - val_acc: 0.7593
Epoch 7/100
596/596 [==============================] - 0s 417us/step - loss: 0.5196 - acc: 0.7768 - val_loss: 0.5055 - val_acc: 0.7559
Epoch 8/100

Epoch 61/100
596/596 [==============================] - 0s 344us/step - loss: 0.4323 - acc: 0.8138 - val_loss: 0.4296 - val_acc: 0.8136
Epoch 62/100
596/596 [==============================] - 0s 353us/step - loss: 0.4338 - acc: 0.8087 - val_loss: 0.4274 - val_acc: 0.8169
Epoch 63/100
596/596 [==============================] - 0s 349us/step - loss: 0.4322 - acc: 0.8138 - val_loss: 0.4262 - val_acc: 0.8068
Epoch 64/100
596/596 [==============================] - 0s 351us/step - loss: 0.4341 - acc: 0.8171 - val_loss: 0.4267 - val_acc: 0.8169
Epoch 65/100
596/596 [==============================] - 0s 351us/step - loss: 0.4317 - acc: 0.8171 - val_loss: 0.4266 - val_acc: 0.8000
Epoch 66/100
596/596 [==============================] - 0s 367us/step - loss: 0.4323 - acc: 0.8070 - val_loss: 0.4251 - val_acc: 0.8203
Epoch 67/100
596/596 [==============================] - 0s 348us/step - loss: 0.4311 - acc: 0.8087 - val_loss: 0.4272 - val_acc: 0.8136
Epoch 68/100
596/596 [==========================

In [72]:
def get_my_x_data(data):
    return [ list([gender_to_int(row['Sex']), row['Age']/max_age, row['Pclass'], (row['SibSp'] + row['Parch'])/mean_n_of_family] for index, row in data.iterrows()) ]
            

test = test.fillna(mean_age)
x_test_data = get_my_x_data(test)

y_pred = model.predict(x_test_data)
y_pred = [1 if x[0] > 0.5 else 0 for x in y_pred]

final = pd.DataFrame(zip(test['PassengerId'], y_pred), columns=['PassengerId', 'Survived'])
final.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [66]:
final.to_csv('result_dl_2.csv', index=False)